In [1]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, lpSum

In [2]:
# datasets 
demand = pd.read_csv("dataset/demand.csv")
vehicles = pd.read_csv("dataset/vehicles.csv")
vehicles_fuels = pd.read_csv("dataset/vehicles_fuels.csv")
fuels = pd.read_csv("dataset/fuels.csv")
carbon_emissions = pd.read_csv("dataset/carbon_emissions.csv")

In [3]:
vehicles = vehicles.merge(vehicles_fuels, on = "ID")

In [4]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          320 non-null    object 
 1   Vehicle                     320 non-null    object 
 2   Size                        320 non-null    object 
 3   Year                        320 non-null    int64  
 4   Cost ($)                    320 non-null    int64  
 5   Yearly range (km)           320 non-null    int64  
 6   Distance                    320 non-null    object 
 7   Fuel                        320 non-null    object 
 8   Consumption (unit_fuel/km)  320 non-null    float64
dtypes: float64(1), int64(3), object(5)
memory usage: 22.6+ KB


In [5]:
model = LpProblem(name = "Fleet_Decarbonization", sense = LpMinimize)

In [6]:
# Varibles 
years = range(2023, 2039)
vehicles_ids = vehicles["ID"].unique()
N_vyr = LpVariable.dicts("Numvehicles", (vehicles_ids, years), lowBound = 0, cat = "Integer")
# Define decision variables for selling vehicles
S_vyr = LpVariable.dicts("SellVehicles", (vehicles_ids, years), lowBound=0, cat='Integer')




In [7]:
# objective function components
C_buy = lpSum(
    [
        N_vyr[v][yr] * vehicles.loc[vehicles["ID"] == v, "Cost ($)"].values[0]
        for v in vehicles_ids
        for yr in years
    ]
)
C_ins = lpSum(
    [
        N_vyr[v][yr] * vehicles.loc[vehicles["ID"] == v, "Cost ($)"].values[0] * 0.1
        for v in vehicles_ids
        for yr in years
    ]
)
C_mnt = lpSum(
    [
        N_vyr[v][yr] * vehicles.loc[vehicles["ID"] == v, "Cost ($)"].values[0] * 0.05
        for v in vehicles_ids
        for yr in years
    ]
)
C_fuel = lpSum(
    [
        N_vyr[v][yr]
        * vehicles.loc[vehicles["ID"] == v, "Consumption (unit_fuel/km)"].values[0]
        * fuels.loc[
            fuels["Fuel"] == vehicles.loc[vehicles["ID"] == v, "Fuel"].values[0],
            "Cost ($/unit_fuel)",
        ].values[0]
        for v in vehicles_ids
        for yr in years
    ]
)
C_sell = lpSum(
    [
        N_vyr[v][yr] * vehicles.loc[vehicles["ID"] == v, "Cost ($)"].values[0] * 0.2
        for v in vehicles_ids
        for yr in years
    ]
)


In [8]:
# total cost

C_total = C_buy + C_ins + C_mnt + C_fuel - C_sell

model += C_total

In [9]:
# Demand constraints: Each year's demand for each size bucket must be met
for year in years:
    for size in demand["Size"].unique():
        model += (
            lpSum(
                [
                    N_vyr[v][year]
                    * vehicles.loc[vehicles["ID"] == v, "Yearly range (km)"].values[0]
                    for v in vehicles_ids
                    if vehicles.loc[vehicles["ID"] == v, "Size"].values[0] == size
                ]
            )
            >= demand[(demand["Year"] == year) & (demand["Size"] == size)][
                "Demand (km)"
            ].sum()
        )

# Carbon emissions constraints: Each year's emissions must not exceed the limit
for year in years:
    model += (
        lpSum(
            [
                N_vyr[v][year]
                * vehicles.loc[vehicles["ID"] == v, "Consumption (unit_fuel/km)"].values[0]
                * fuels.loc[
                    fuels["Fuel"]
                    == vehicles.loc[vehicles["ID"] == v, "Fuel"].values[0],
                    "Emissions (CO2/unit_fuel)",
                ].values[0]
                for v in vehicles_ids
            ]
        )
        <= carbon_emissions.loc[
            carbon_emissions["Year"] == year, "Carbon emission CO2/kg"
        ].values[0]
    )

# Vehicles bought in year t are available for use up to year t+10
for v in vehicles_ids:
    for year in range(2023, 2029):
        model += (lpSum([N_vyr[v][yr] for yr in range(year, year+11)]) - lpSum([S_vyr[v][yr] for yr in range(year+10, year+11)]) >= 0)

# Ensure that we sell the vehicles exactly after 10 years of use
for v in vehicles_ids:
    for year in range(2033, 2039):
        model += (S_vyr[v][year] == N_vyr[v][year-10])


In [10]:
model.solve()

1

In [18]:
# Extract results and save to CSV
results = []

for v in vehicles_ids:
    for yr in years:
        if N_vyr[v][yr].varValue is not None:
            if N_vyr[v][yr].varValue > 0:
                results.append([yr, v, int(N_vyr[v][yr].varValue), 'Buy', None, None, 0])
        if S_vyr[v][yr].varValue is not None:
            if S_vyr[v][yr].varValue > 0:
                results.append([yr, v, int(S_vyr[v][yr].varValue), 'Sell', None, None, 0])


results_df = pd.DataFrame(
    results,
    columns=[
        "Year",
        "ID",
        "Num_Vehicles",
        "Type",
        "Fuel",
        "Distance_bucket",
        "Distance_per_vehicle(km)",
    ],
)
results_df.to_csv("solution.csv", index=False)
